In [ ]:
from testplan import Testplan
from testplan.common.utils.logger import TEST_INFO, DEBUG
from my_tests.mtest import make_multitest

In [ ]:
# Initialize a plan with interactive mode flag set.
plan = Testplan(name='MyPlan',
                interactive=True,
                parse_cmdline=False,
                logger_level=TEST_INFO)

In [ ]:
# Interactive mode serving interactive requests.
ihandler = plan.run()

In [ ]:
# Adding a test.
test1_uid = plan.add(make_multitest(idx='1'))
print('Test uid: {}'.format(test1_uid))

In [ ]:
print('Testplan interactive handler: '.format(plan.i))
test = plan.i.test(test1_uid)
print('Test added: {}'.format(test))
print('Test environment drivers: {}'.format(test))

In [ ]:
# Start the test envirorment resources (Server & client).
plan.i.start_test_resources(test_uid='Test1')

In [ ]:
print('Server status: {}'.format(test.resources.server.status.tag))
print('Client status: {}'.format(test.resources.client.status.tag))

In [ ]:
# Run a test case.
plan.i.run_test_case(test_uid=test1_uid, suite_uid='TCPSuite', case_uid='send_and_receive_msg')

In [ ]:
# Stop test resources.
plan.i.stop_test_resources(test_uid='Test1')

In [ ]:
print('Server status: {}'.format(test.resources.server.status.tag))
print('Client status: {}'.format(test.resources.client.status.tag))

In [ ]:
# Add an independent environment.
from testplan.environment import LocalEnvironment
from testplan.testing.multitest.driver.tcp import TCPServer, TCPClient
from testplan.common.utils.context import context

plan.add_environment(
    LocalEnvironment(
        'my_env1',
        [TCPServer(name='server'),
         TCPClient(name='client',
                   host=context('server', '{{host}}'),
                   port=context('server', '{{port}}'))]))

In [ ]:
# Operate my_env1
env1 = plan.i.get_environment('my_env1')
env1.start()
env1.server.accept_connection()
print('Client sends msg of length in bytes: {}'.format(
        env1.client.send_text('Hello server!')))
print('Server receives: {}'.format(
        env1.server.receive_text()))
env1.stop()

In [ ]:
# You can add an environment using HTTP requests from another tool i.e a UI.
# To demonstrate that:
import requests

addr = 'http://{}:{}'.format(*plan.i.http_handler_info)
print('HTTP listener: {}'.format(addr))
response = requests.post('{}/sync/create_new_environment'.format(addr),
                         json={'env_uid': 'my_env2'})
response = requests.post('{}/sync/add_environment_resource'.format(addr),
                         json={'env_uid': 'my_env2',
                               'target_class_name': 'TCPServer',
                               'name': 'server'})
response = requests.post('{}/sync/add_environment_resource'.format(addr),
                         json={'env_uid': 'my_env2',
                               'target_class_name': 'TCPClient',
                               'name': 'client',
                               '_ctx_host_ctx_driver': 'server',
                               '_ctx_host_ctx_value': '{{host}}',
                               '_ctx_port_ctx_driver': 'server',
                               '_ctx_port_ctx_value': '{{port}}'})
response = requests.post('{}/sync/add_created_environment'.format(addr),
                         json={'env_uid': 'my_env2'})

In [ ]:
# Operate my_env1 using HTTP.

# Start and initialize connection.
response = requests.post('{}/sync/start_environment'.format(addr),
                         json={'env_uid': 'my_env2'})
response = requests.post('{}/sync/environment_resource_operation'.format(addr),
                         json={'env_uid': 'my_env2',
                               'resource_uid': 'server',
                               'operation': 'accept_connection'})

# Drivers operations example - send and receive a message.
msg = 'Hello world'
response = requests.post('{}/sync/environment_resource_operation'.format(addr),
                         json={'env_uid': 'my_env2',
                               'resource_uid': 'client',
                               'operation': 'send_text',
                             'msg': msg})
response = requests.post('{}/sync/environment_resource_operation'.format(addr),
                         json={'env_uid': 'my_env2',
                         'resource_uid': 'server',
                         'operation': 'receive_text'})
print('Servers receives: {}'.format(response.json()['result']))

# Stop the environment.
response = requests.post('{}/sync/stop_environment'.format(addr),
                         json={'env_uid': 'my_env2'})

In [ ]:
print('Independent environments added: {}'.format(
        list(plan.resources.environments.envs.keys())))

In [ ]:
# Abort the plan.
plan.abort()